In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# Load libraries
from sklearn.dummy import DummyClassifier
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

In [2]:
# Read the CSV file using Pandas.
alldata = pd.read_csv('sberbank.csv')

# Convert the timestamp string to an integer representing the year.
def get_year(timestamp):
    return int(timestamp[:4])
alldata['year'] = alldata.timestamp.apply(get_year)

# Select the 9 input columns and the output column.
selected_columns = ['price_doc', 'year', 'full_sq', 'life_sq', 'floor', 'num_room', 'kitch_sq', 'full_all']
alldata = alldata[selected_columns]
alldata = alldata.dropna()

# Shuffle.
alldata_shuffled = alldata.sample(frac=1.0, random_state=0)

# Separate the input and output columns.
X = alldata_shuffled.drop('price_doc', axis=1)
# For the output, we'll use the log of the sales price.
Y = alldata_shuffled['price_doc'].apply(np.log)

# Split into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
m1 = DummyRegressor()
cross_validate(m1, Xtrain, Ytrain, scoring='neg_mean_squared_error')

{'fit_time': array([0.00600123, 0.00300765, 0.00454187, 0.00299406, 0.00152898]),
 'score_time': array([0.00450778, 0.00099087, 0.00102234, 0.00099897, 0.00099945]),
 'test_score': array([-0.39897319, -0.37113485, -0.38083108, -0.39057156, -0.40475168])}

In [25]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [13]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [29]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}


In [30]:
rf_Model = GradientBoostingRegressor()

In [31]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = rf_Model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [32]:
rf_Grid.fit(Xtrain, Ytrain)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=4,
             param_grid={'max_depth': [2, 4], 'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [33]:
rf_Grid.best_params_

{'max_depth': 4,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 80}

In [35]:
print (f'Train Accuracy - : {rf_Grid.score(Xtrain,Ytrain):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(Xtrain,Ytrain):.3f}')

Train Accuracy - : 0.369
Test Accuracy - : 0.369


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [12]:
# grid search
parameters = {
    'max_depth': [1, 2, 3, 4,5,6,7, 8,9,10,15,20,30,40,50],
     "min_samples_split": np.linspace(0.001, 0.5, 12),
     "min_samples_leaf": np.linspace(0.001, 0.5, 10),
    'criterion':['gini','entropy'],
    "max_features":["log2","sqrt"],
}

#passing the scoring function in the GridSearchCV
clf = GridSearchCV(RandomForestClassifier(), parameters,  refit=False,cv=2, n_jobs=-1) # scoring=scoring,

clf.fit(Xtrain, Ytrain)

print ("Best Score: {}".format(clf.best_score_))
print ("Best params: {}".format(clf.best_params_))

Best Score: nan
Best params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'log2', 'min_samples_leaf': 0.001, 'min_samples_split': 0.001}
